Collecting & Cleaning Data:

Create bar chart across 8 years plotting monthly totals of each crime (layered, so that the highest number is behind the middle and smallest number theft)

*Remove API code

In [1]:
import pandas as pd
import numpy as np
import googlemaps

In [2]:
garage = pd.read_csv('Irvine Garage Burglary.csv')

In [3]:
garage.head(1)

,DR,Offense,Date_Rept,Monthstamp,Yearstamp,Block
0,1100198,BURGLARY - RESIDENTIAL - GARAGE ...,1/5/2011,1,2011,0 Blk REMINGTON ...


In [4]:
vehicle = pd.read_csv('Irvine Vehicle Theft.csv')

In [5]:
vehicle.head(1)

,DR,Offense,Date_Rept,Monthstamp,Yearstamp,Block
0,1100034,THEFT - FROM MOTOR VEHICLE ...,1/1/2011,1,2011,CAPSTONE / RIVERSTONE ...


In [6]:
package = pd.read_csv('Irvine Package Theft.csv')

In [7]:
package.head(1)

,DR,Date_Rept,Monthstamp,Yearstamp,Block
0,1607426,5/12/2016,5,2016,100 Blk ESPLANADE


In [8]:
# Removing so all files have identical columns
garage = garage.drop('Offense', axis='columns')
vehicle = vehicle.drop('Offense', axis='columns')

In [9]:
garage.head(1)

,DR,Date_Rept,Monthstamp,Yearstamp,Block
0,1100198,1/5/2011,1,2011,0 Blk REMINGTON ...


In [10]:
vehicle.head(1)

,DR,Date_Rept,Monthstamp,Yearstamp,Block
0,1100034,1/1/2011,1,2011,CAPSTONE / RIVERSTONE ...


---

#### Adding 'Type' Column :
(so rows are distinguishable when files are joined)

In [11]:
files = [garage, vehicle, package]

In [12]:
file_strings = ['Garage', 'Vehicle', 'Package']
counter = 0
for file in files:
    type_col = np.full(len(file), file_strings[counter])
    file['Type'] = type_col
    counter += 1

In [13]:
combined = pd.concat([garage, vehicle, package])

In [14]:
combined.Date_Rept = pd.to_datetime(combined.Date_Rept)

In [15]:
combined = combined.sort_values('Date_Rept').reset_index(drop=True)

In [16]:
combined['Week_Num'] = combined.Date_Rept.dt.week

In [17]:
# To reveal all data in each cell (instead of cutting off at default max character length)
pd.set_option('display.max_colwidth', -1)

In [18]:
combined.head()

,DR,Date_Rept,Monthstamp,Yearstamp,Block,Type,Week_Num
0,1100034,2011-01-01,1,2011,CAPSTONE / RIVERSTONE,Vehicle,52
1,1100051,2011-01-02,1,2011,14800 Blk GAINFORD CIR,Vehicle,52
2,1100108,2011-01-04,1,2011,100 Blk SAINT VINCENT,Vehicle,1
3,1100198,2011-01-05,1,2011,0 Blk REMINGTON,Garage,1
4,1100212,2011-01-05,1,2011,2800 Blk ALTON PKWY,Vehicle,1


In [19]:
combined.loc[((combined.Monthstamp == 1) & (combined.Week_Num > 51)), 'Week_Num'] = 1

In [20]:
combined.head()

,DR,Date_Rept,Monthstamp,Yearstamp,Block,Type,Week_Num
0,1100034,2011-01-01,1,2011,CAPSTONE / RIVERSTONE,Vehicle,1
1,1100051,2011-01-02,1,2011,14800 Blk GAINFORD CIR,Vehicle,1
2,1100108,2011-01-04,1,2011,100 Blk SAINT VINCENT,Vehicle,1
3,1100198,2011-01-05,1,2011,0 Blk REMINGTON,Garage,1
4,1100212,2011-01-05,1,2011,2800 Blk ALTON PKWY,Vehicle,1


In [21]:
combined = combined.rename({'Date_Rept':'Date', 'Monthstamp':'Month', 'Yearstamp':'Year'}, axis=1)

In [22]:
combined.tail()

,DR,Date,Month,Year,Block,Type,Week_Num
6385,1907102,2019-05-28,5,2019,1400 Blk PENDIO,Garage,22
6386,1907155,2019-05-29,5,2019,100 Blk RINALDI,Vehicle,22
6387,1907159,2019-05-29,5,2019,0 Blk PLANTATION,Vehicle,22
6388,1907231,2019-05-30,5,2019,400 Blk ROCKEFELLER,Vehicle,22
6389,1907311,2019-05-31,5,2019,0 Blk OAK TREE LN,Vehicle,22


---

In [23]:
combined.Year.value_counts().sort_index(ascending=False)

2019    431
2018    985
2017    880
2016    915
2015    915
2014    493
2013    556
2012    619
2011    596
Name: Year, dtype: int64

---

In [24]:
# Address Number only
combined.Block.str.extract('(\d+)').fillna(0).head()

,0
0,0
1,14800
2,100
3,0
4,2800


In [25]:
combined['Number'] = combined.Block.str.extract('(\d+)').fillna(0).astype(str)

In [26]:
# Street name only
combined.Block.str.extract('([^\d+]+)').head()

,0
0,CAPSTONE / RIVERSTONE
1,Blk GAINFORD CIR
2,Blk SAINT VINCENT
3,Blk REMINGTON
4,Blk ALTON PKWY


In [27]:
# First object in series (instead of df- via "expand=False")
combined.Block.str.extract('([^\d+]+)', expand=False)[0]

'CAPSTONE / RIVERSTONE                                                           '

In [28]:
combined.Block.str.extract('([^\d+]+)', expand=False)[1]

' Blk GAINFORD CIR                                                                    '

In [29]:
street_name = combined.Block.str.extract('([^\d+]+)', expand=False)

In [30]:
street_name[0]

'CAPSTONE / RIVERSTONE                                                           '

In [31]:
street_name[1]

' Blk GAINFORD CIR                                                                    '

In [32]:
# Removing whitespance & ' Blk'
combined['Street'] = [address.strip()[4:] if address[:4] == ' Blk' else address.strip() for address in street_name]

In [33]:
combined['Street'][1]

'GAINFORD CIR'

---

In [34]:
combined['Address'] = combined.Number + ' ' + combined.Street + ', Irvine, CA'

In [35]:
combined.head()

,DR,Date,Month,Year,Block,Type,Week_Num,Number,Street,Address
0,1100034,2011-01-01,1,2011,CAPSTONE / RIVERSTONE,Vehicle,1,0,CAPSTONE / RIVERSTONE,"0 CAPSTONE / RIVERSTONE, Irvine, CA"
1,1100051,2011-01-02,1,2011,14800 Blk GAINFORD CIR,Vehicle,1,14800,GAINFORD CIR,"14800 GAINFORD CIR, Irvine, CA"
2,1100108,2011-01-04,1,2011,100 Blk SAINT VINCENT,Vehicle,1,100,SAINT VINCENT,"100 SAINT VINCENT, Irvine, CA"
3,1100198,2011-01-05,1,2011,0 Blk REMINGTON,Garage,1,0,REMINGTON,"0 REMINGTON, Irvine, CA"
4,1100212,2011-01-05,1,2011,2800 Blk ALTON PKWY,Vehicle,1,2800,ALTON PKWY,"2800 ALTON PKWY, Irvine, CA"


*Google maps seems to accurately identify addresses w/ current column format (e.g. slashes, missing numbers); I will probably only have to correct for misspellings

---

Package Theft Addresses:

In [36]:
API_KEY = (Hidden)

In [37]:
googmap = googlemaps.Client(key=API_KEY)

In [38]:
# Took about 45 minutes to complete
lats = []
lons = []

for address in combined.Address:
    geocode = googmap.geocode(address)
    try:
        lats.append(float(geocode[0]['geometry']['location']['lat']))
        lons.append(float(geocode[0]['geometry']['location']['lng']))
    except:
        lats.append(None)
        lons.append(None)

In [39]:
len(lats)

6390

In [40]:
combined['Lat'] = lats
combined['Lon'] = lons

In [41]:
combined.head()

,DR,Date,Month,Year,Block,Type,Week_Num,Number,Street,Address,Lat,Lon
0,1100034,2011-01-01,1,2011,CAPSTONE / RIVERSTONE,vehicle,1,0,CAPSTONE / RIVERSTONE,"0 CAPSTONE / RIVERSTONE, Irvine, CA",33.700425,-117.796927
1,1100051,2011-01-02,1,2011,14800 Blk GAINFORD CIR,vehicle,1,14800,GAINFORD CIR,"14800 GAINFORD CIR, Irvine, CA",33.694986,-117.783549
2,1100108,2011-01-04,1,2011,100 Blk SAINT VINCENT,vehicle,1,100,SAINT VINCENT,"100 SAINT VINCENT, Irvine, CA",33.666169,-117.789836
3,1100198,2011-01-05,1,2011,0 Blk REMINGTON,garage,1,0,REMINGTON,"0 REMINGTON, Irvine, CA",33.700298,-117.769388
4,1100212,2011-01-05,1,2011,2800 Blk ALTON PKWY,vehicle,1,2800,ALTON PKWY,"2800 ALTON PKWY, Irvine, CA",33.687964,-117.831472


---

### Checking for Missing or Incorrect Lat/Lon

In [42]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6390 entries, 0 to 6389
Data columns (total 12 columns):
DR          6390 non-null int64
Date        6390 non-null datetime64[ns]
Month       6390 non-null int64
Year        6390 non-null int64
Block       6390 non-null object
Type        6390 non-null object
Week_Num    6390 non-null int64
Number      6390 non-null object
Street      6390 non-null object
Address     6390 non-null object
Lat         6389 non-null float64
Lon         6389 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 599.2+ KB


---

#### (1) Filling in Missing lat/lon:

In [43]:
combined[combined.Lon.isna()]

,DR,Date,Month,Year,Block,Type,Week_Num,Number,Street,Address,Lat,Lon
1441,1305130,2013-04-26,4,2013,0 Blk ISLANDVIEW,vehicle,17,0,ISLANDVIEW,"0 ISLANDVIEW, Irvine, CA",NaN,NaN


In [44]:
# Typo- should be a space b/w "Island" & "View"
combined.loc[combined.Address == '0 ISLANDVIEW, Irvine, CA', 'Address'] = '0 ISLAND VIEW, Irvine, CA'

In [45]:
combined[combined.Address == '0 ISLAND VIEW, Irvine, CA']

,DR,Date,Month,Year,Block,Type,Week_Num,Number,Street,Address,Lat,Lon
1441,1305130,2013-04-26,4,2013,0 Blk ISLANDVIEW,vehicle,17,0,ISLANDVIEW,"0 ISLAND VIEW, Irvine, CA",NaN,NaN


In [46]:
geocode = googmap.geocode('0 ISLAND VIEW, Irvine, CA')
combined.loc[combined.Address == '0 ISLAND VIEW, Irvine, CA', 'Lat'] = float(geocode[0]['geometry']['location']['lat'])
combined.loc[combined.Address == '0 ISLAND VIEW, Irvine, CA', 'Lon'] = float(geocode[0]['geometry']['location']['lng'])

In [47]:
combined[combined.Address == '0 ISLAND VIEW, Irvine, CA']

,DR,Date,Month,Year,Block,Type,Week_Num,Number,Street,Address,Lat,Lon
1441,1305130,2013-04-26,4,2013,0 Blk ISLANDVIEW,vehicle,17,0,ISLANDVIEW,"0 ISLAND VIEW, Irvine, CA",33.682307,-117.795601


In [48]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6390 entries, 0 to 6389
Data columns (total 12 columns):
DR          6390 non-null int64
Date        6390 non-null datetime64[ns]
Month       6390 non-null int64
Year        6390 non-null int64
Block       6390 non-null object
Type        6390 non-null object
Week_Num    6390 non-null int64
Number      6390 non-null object
Street      6390 non-null object
Address     6390 non-null object
Lat         6390 non-null float64
Lon         6390 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 599.2+ KB


In [49]:
# Examine plotting of coordinates on map (via Mapbox)- look for errors
combined.to_csv('all_irvine_thefts.csv', index=False)

---

#### (2) Correcting Incorrect Lat/Lon Coordinates:

##### *Examining Points from all_irvine_thefts.csv on Map (in Mapbox- datasets)
(points lying outside Irvine, CA boundaries will be manually assigned correct lat/lon coordinates)

In [50]:
combined[combined.Address.str.contains('EL CAMINO REAL')]

,DR,Date,Month,Year,Block,Type,Week_Num,Number,Street,Address,Lat,Lon
2047,1409643,2014-08-01,8,2014,4600 Blk EL CAMINO REAL,Package,31,4600,EL CAMINO REAL,"4600 EL CAMINO REAL, Irvine, CA",35.496727,-120.681378
2425,1504171,2015-03-26,3,2015,4500 Blk EL CAMINO REAL,Vehicle,13,4500,EL CAMINO REAL,"4500 EL CAMINO REAL, Irvine, CA",37.401606,-122.115302
3868,1614520,2016-09-22,9,2016,4300 Blk EL CAMINO REAL,Package,38,4300,EL CAMINO REAL,"4300 EL CAMINO REAL, Irvine, CA",37.405366,-122.119734
4519,1709619,2017-07-03,7,2017,4700 Blk EL CAMINO REAL,Vehicle,27,4700,EL CAMINO REAL,"4700 EL CAMINO REAL, Irvine, CA",37.399946,-122.112171
5822,1816277,2018-11-23,11,2018,4500 Blk EL CAMINO REAL,Vehicle,47,4500,EL CAMINO REAL,"4500 EL CAMINO REAL, Irvine, CA",37.401606,-122.115302


In [51]:
# The new lat/lon coordinates below are approximately correct 
for i, row in combined[combined.Address.str.contains('EL CAMINO REAL')].iterrows():
    combined.loc[i, 'Lat'] = 33.717191
    combined.loc[i, 'Lon'] = -117.784642

In [52]:
combined[combined.Address.str.contains('EL CAMINO REAL')]

,DR,Date,Month,Year,Block,Type,Week_Num,Number,Street,Address,Lat,Lon
2047,1409643,2014-08-01,8,2014,4600 Blk EL CAMINO REAL,Package,31,4600,EL CAMINO REAL,"4600 EL CAMINO REAL, Irvine, CA",33.717191,-117.784642
2425,1504171,2015-03-26,3,2015,4500 Blk EL CAMINO REAL,Vehicle,13,4500,EL CAMINO REAL,"4500 EL CAMINO REAL, Irvine, CA",33.717191,-117.784642
3868,1614520,2016-09-22,9,2016,4300 Blk EL CAMINO REAL,Package,38,4300,EL CAMINO REAL,"4300 EL CAMINO REAL, Irvine, CA",33.717191,-117.784642
4519,1709619,2017-07-03,7,2017,4700 Blk EL CAMINO REAL,Vehicle,27,4700,EL CAMINO REAL,"4700 EL CAMINO REAL, Irvine, CA",33.717191,-117.784642
5822,1816277,2018-11-23,11,2018,4500 Blk EL CAMINO REAL,Vehicle,47,4500,EL CAMINO REAL,"4500 EL CAMINO REAL, Irvine, CA",33.717191,-117.784642


---

In [53]:
# Creating column corresponding to 'Type' that contains numeric values for each type (to manipulate color in mapbox)
combined['Type_Num'] = combined.Type.map({'Vehicle':0,'Package':1, 'Garage':2})

In [54]:
combined.head()

,DR,Date,Month,Year,Block,Type,Week_Num,Number,Street,Address,Lat,Lon,Type_Num
0,1100034,2011-01-01,1,2011,CAPSTONE / RIVERSTONE,Vehicle,1,0,CAPSTONE / RIVERSTONE,"0 CAPSTONE / RIVERSTONE, Irvine, CA",33.700425,-117.796927,0
1,1100051,2011-01-02,1,2011,14800 Blk GAINFORD CIR,Vehicle,1,14800,GAINFORD CIR,"14800 GAINFORD CIR, Irvine, CA",33.694986,-117.783549,0
2,1100108,2011-01-04,1,2011,100 Blk SAINT VINCENT,Vehicle,1,100,SAINT VINCENT,"100 SAINT VINCENT, Irvine, CA",33.666169,-117.789836,0
3,1100198,2011-01-05,1,2011,0 Blk REMINGTON,Garage,1,0,REMINGTON,"0 REMINGTON, Irvine, CA",33.700298,-117.769388,2
4,1100212,2011-01-05,1,2011,2800 Blk ALTON PKWY,Vehicle,1,2800,ALTON PKWY,"2800 ALTON PKWY, Irvine, CA",33.687964,-117.831472,0


In [55]:
combined.to_csv('all_irvine_thefts_final.csv', index=False)